In [ ]:
!pip install langchain tiktoken pypdf sentence_transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
                  chunk_size=500,
                  chunk_overlap=50,
                  length_function=tiktoken_len
)

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf')
pages = loader.load_and_split()

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name='jhgan/ko-sbert-nli'
model_kwargs={'device':'cpu'}
encode_kwargs={'normalize_embeddings':True}

hf = HuggingFaceEmbeddings(
        model_name = model_name,
        model_kwargs = model_kwargs,
        encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 데이터의 Split
docs = text_splitter.split_documents(pages)

# Vector Store 저장
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, hf)

In [ ]:
query = '탄소중립'
docs_result = db.similarity_search(query)

In [ ]:
print(docs_result[0].page_content)
print('-' * 100)
print(docs_result[0].metadata['page'])

01이노비즈 정책브리프
01 탄소중립과 대응 필요성
탄소중립이란 ?
∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제
로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함
- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로
(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음
*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수
소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 
대응 필요성
∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가
운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨
----------------------------------------------------------------------------------------------------
3


In [ ]:
docs_result

[Document(page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨', metadata={'page': 3, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf'}),
 Document(page_content='보제공 확대 \n- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제\n공 확대 등 순환 사회로의 전환 가속화\n*탄소발자국이란 개인 또는 기업, 국가 등의 활동이나 상품을 생산하고 소비하는 전체 과정을 통해 발생시키는 온실가스 , 특히 이산화탄소의 \n총량을 의미함\n ③ 탄소중립 사회로의 공정전환\n- (취약 산업·계층 보호) 내연기관차 완성차 및 부품업체 등 축소산업에 대한 R&D, M&A 등을 통해 대\n체·유망분야로 사업전환 적극 지원, 맞춤형 재취업 지원\n- 구조전환으로 축소되는 석탄발전 ‧내연

In [ ]:
docs_result = db.similarity_search_with_relevance_scores(query, k=3)
len(docs_result)

3

In [ ]:
docs_result

[(Document(page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨', metadata={'page': 3, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf'}),
  0.6235679934827092),
 (Document(page_content='보제공 확대 \n- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제\n공 확대 등 순환 사회로의 전환 가속화\n*탄소발자국이란 개인 또는 기업, 국가 등의 활동이나 상품을 생산하고 소비하는 전체 과정을 통해 발생시키는 온실가스 , 특히 이산화탄소의 \n총량을 의미함\n ③ 탄소중립 사회로의 공정전환\n- (취약 산업·계층 보호) 내연기관차 완성차 및 부품업체 등 축소산업에 대한 R&D, M&A 등을 통해 대\n체·유망분야로 사업전환 적극 지원, 맞춤형 재취업 지

In [ ]:
db = Chroma.from_documents(docs, hf, persist_directory='./chroma_db')

In [ ]:
db2 = Chroma(persist_directory='./chroma_db',
             embedding_function=hf)

In [ ]:
db2.similarity_search(query)

[Document(page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨', metadata={'page': 3, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf'}),
 Document(page_content='보제공 확대 \n- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제\n공 확대 등 순환 사회로의 전환 가속화\n*탄소발자국이란 개인 또는 기업, 국가 등의 활동이나 상품을 생산하고 소비하는 전체 과정을 통해 발생시키는 온실가스 , 특히 이산화탄소의 \n총량을 의미함\n ③ 탄소중립 사회로의 공정전환\n- (취약 산업·계층 보호) 내연기관차 완성차 및 부품업체 등 축소산업에 대한 R&D, M&A 등을 통해 대\n체·유망분야로 사업전환 적극 지원, 맞춤형 재취업 지원\n- 구조전환으로 축소되는 석탄발전 ‧내연

# FAISS

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 45.3 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, hf)

In [ ]:
db.similarity_search(query)

[Document(page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨', metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf', 'page': 3}),
 Document(page_content='보제공 확대 \n- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제\n공 확대 등 순환 사회로의 전환 가속화\n*탄소발자국이란 개인 또는 기업, 국가 등의 활동이나 상품을 생산하고 소비하는 전체 과정을 통해 발생시키는 온실가스 , 특히 이산화탄소의 \n총량을 의미함\n ③ 탄소중립 사회로의 공정전환\n- (취약 산업·계층 보호) 내연기관차 완성차 및 부품업체 등 축소산업에 대한 R&D, M&A 등을 통해 대\n체·유망분야로 사업전환 적극 지원, 맞춤형 재취업 지원\n- 구조전환으로 축소되는 석탄발전 ‧내연